In [1]:
import sys
import os
import miniupnpc
from pyproj import Proj, transform
from fabric import Connection

#adding prognos tools
sys.path.insert(0, "/home/jose-luis/Envs/niva_prognos/PROGNOS/")
from prognos_tools.encrypt import decryptString,encryptString
from prognos_tools.Basin_fabric2 import Basin

import psycopg2
import getpass
import gmaps

gmapsToken=b'gAAAAABctxJp5wE73qK6U5VieCi0WXNnNex4KxLZHutsESa8fW9v43lLa1Ag0qsxKFjXXo9MBAdvUpqJPW-QmCE0gH_Opf9g4xAG1VaI2WarO_xDZg44VLMCHkd_6O8ofgp8u4VuFBMr'
key = getpass.getpass('Password: ')
apiKey = decryptString(gmapsToken,key)
gmaps.configure(api_key=apiKey)
del key,apiKey

Password:  ·······


# Creation of database containing processed dem for catchment delineation
## Creation of virtual machine with postgresql, postgis and plpgsql

In [2]:
cloudInfo = {'project'      : 'nivacatchment',
             'zone'         : 'europe-west3-a',
             'instanceType' : "n1-highmem-16",
             'instanceName' : "svalbard",
             'image'        : 'debian-9-stretch-v20190423',
             'imageProject' : 'debian-cloud',
             'diskSize'     : '200',
             'username'     : "jose-luis",
             'keyDir'       : './'
             }




instantiationDict= {
  "machineType" : "zones/{}/machineTypes/{}".format(cloudInfo['zone'],cloudInfo['instanceType']),
  "name" : cloudInfo['instanceName'],
  "canIpForward": "false",
  "networkInterfaces": [
     {
       "subnetwork": "projects/{}/regions/{}/subnetworks/default".format(cloudInfo['project'], '-'.join(cloudInfo['zone'].split('-')[:-1]) ),
       "accessConfigs": [
        {
          "kind": "compute#networkInterface",
          "name": "External NAT",
          "type": "ONE_TO_ONE_NAT",
          "networkTier": "PREMIUM"
        }
       ]     
     }
     ],
    
 "tags": {
    "items": [
      "http-server","https-server","postgres","ssh"
    ]
  },
  "disks": [
    {
      "boot": True,
      "autoDelete": True,
      "deviceName": cloudInfo['instanceName'],
      "initializeParams": {
        "sourceImage": "projects/debian-cloud/global/images/debian-9-stretch-v20191014",
#         "diskType": "projects/{}/zones/{}/diskTypes/pd-standard".format(gce.project,gce.zone),
         "diskSizeGb": cloudInfo['diskSize']
      }
    }
  ],
  "metadata": {
    "items": [
      {
       "key": "ssh-keys",
       "value": "dummy"
      },
      {
       "key": "startup-script",
       "value": '''#!/bin/bash
sudo apt-get update
yes | sudo apt-get install postgresql postgresql-contrib postgis postgresql-server-dev-all unzip git cmake g++ gcc libopenmpi-dev gdal-bin libgdal-dev python-gdal libfftw3-dev htop grass grass-dev libgdal-grass saga

export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s`
echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

yes | sudo apt-get update
yes | sudo apt-get install gcsfuse

'''
      },
    ]
  } 
}


# "initializeParams": {
#        "sourceImage": "projects/debian-cloud/global/images/debian-9-stretch-v20191014",
#        "diskType": "projects/nivacatchment/zones/us-central1-a/diskTypes/pd-standard",
#        "diskSizeGb": "10"
#      },

In [3]:
#Vm characteristics
basin=Basin('/home/jose-luis/Envs/gce_framework/code/keys/nivacatchment.json',cloudInfo)
display(basin.properties)

info = basin.get('projectInfo')
display('Can now talk to project {}'.format(info['name']))


#Setting ssh firewall for project
inOffice = True
if not inOffice:    
    with Connection('localhost') as c:
        hubIp = c.local('curl https://ipinfo.io/ip').stdout.strip()
        basin.setSSHPort(hubIp,inOffice=False)
        basin.setPostgresAccess(hubIp,inOffice=False)
else:
    basin.setSSHPort()
    basin.setPostgresAccess()
    
#Actually instantiating the machine
ip=basin.instantiate(wait=True,ownDict=True,inst=instantiationDict) #wait until the vm is actually up and running
display(ip)

#Allowing local connections on the VM without password by modifying the pg_hba.conf file
with Connection(host=basin.properties['ip'],
                user=basin.properties['username'],
                connect_kwargs={"key_filename": basin.properties['keyFile'],}
                ) as c:
    c.sudo('''find /etc -name pg_hba.conf -exec sed -i 's|\(^host \+all \+all \+::1/128 \+\).*|\\1trust|g' {} \; ''')
    c.sudo("service postgresql restart")


#Defining custom queries to geodatabase. Note that we forwarded the port on the VM to the
#local 5432 port so as to be able to talk to postgresql through the local port
def query(query,fetch=True):
    with Connection(host=basin.properties['ip'],
                    user=basin.properties['username'],
                    connect_kwargs={"key_filename": basin.properties['keyFile']}).forward_local(5432):
        db = psycopg2.connect(host='localhost', port=5432, database='geosvalbard')
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            return(cursor.fetchall()) 

#Setting up the connection    
basin.setConnection()

['Oct 29 12:34:33 svalbard systemd[1]: Startup finished in 2.096s (kernel) + 2min 10.853s (userspace) = 2min 12.950s.']

'34.89.150.135'

In [4]:
svalbardScript = '''#!/bin/bash

git clone https://github.com/petere/plsh.git
cd plsh
make
sudo make install
cd ..

#Allowing ssh connection to server
sudo find /etc -name postgresql.conf -exec sudo sed -i s/.*listen_addresses.*/listen_addresses\\ =\\ \\'*\\'/g {} \;
sudo find /etc -name pg_hba.conf -exec sh -c "echo host all all 151.157.0.0/16 md5>> {}" \;
sudo service postgresql restart

#Creating user and geosvalbard database
sudo -u postgres createuser jose-luis
sudo -u postgres createdb geosvalbard
a=\\\\\\\"jose-luis\\\\\\\"
sudo su -c "psql -d geosvalbard -c \\"grant all privileges on database geosvalbard to $a;\\"" postgres
sudo su -c "psql -d geosvalbard -c \\"alter user $a with superuser;\\"" postgres
sudo su -c "psql -d geosvalbard -c \\"alter user $a with password 'kakaroto';\\"" postgres

#enable postgis on database
echo "ALTER DATABASE geosvalbard SET search_path=public, postgis, contrib, topology;" | psql -d geosvalbard
echo "CREATE EXTENSION postgis CASCADE;" | psql -d geosvalbard
echo "CREATE EXTENSION postgis_topology CASCADE;" | psql -d geosvalbard
echo "ALTER DATABASE geosvalbard SET search_path=public, postgis, contrib, topology;" | psql -d geosvalbard
echo "ALTER DATABASE geosvalbard SET postgis.gdal_enabled_drivers = ENABLE_ALL;" | psql -d geosvalbard
echo "SELECT pg_reload_conf();" | psql -d geosvalbard
echo "SET postgis.enable_outdb_rasters TO True;" | psql -d geosvalbard
sudo service postgresql restart
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)

basin.connection.put('script.sh')
basin.connection.run('chmod +x script.sh')
basin.connection.run('./script.sh')
basin.connection.run('rm script.sh')


Cloning into 'plsh'...


gcc -Wall -Wpointer-arith -Wdeclaration-after-statement -Wendif-labels -Wmissing-format-attribute -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -g -g -O2 -fdebug-prefix-map=/build/postgresql-9.6-Dk87xT/postgresql-9.6-9.6.15=. -specs=/usr/share/dpkg/no-pie-compile.specs -fstack-protector-strong -Wformat -Werror=format-security -I/usr/include/mit-krb5 -fPIC -pie -fno-omit-frame-pointer -fPIC -I. -I./ -I/usr/include/postgresql/9.6/server -I/usr/include/postgresql/internal -Wdate-time -D_FORTIFY_SOURCE=2 -D_GNU_SOURCE -I/usr/include/libxml2  -I/usr/include/tcl8.6  -c -o plsh.o plsh.c
gcc -Wall -Wpointer-arith -Wdeclaration-after-statement -Wendif-labels -Wmissing-format-attribute -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -g -g -O2 -fdebug-prefix-map=/build/postgresql-9.6-Dk87xT/postgresql-9.6-9.6.15=. -specs=/usr/share/dpkg/no-pie-compile.specs -fstack-protector-strong -Wformat -Werror=format-security -I/usr/include/mit-krb5 -fP

<Result cmd='rm script.sh' exited=0>

## Downloading elevation and river datasets

In [5]:
#Extracting a subset of the raster (around Adventelva)
adventelvaY,adventelvaX = 78.20486,15.81774
#Converting to epsg:3035
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3035')
x,y = transform(inProj,outProj,adventelvaX,adventelvaY)
offset=120000
box={'ulx': x - offset, 'uly': y + offset , 'lrx': x + offset , 'lry': y - offset }

svalbardScript='''#!/bin/bash
#Getting 20m DEM
wget -q http://publicdatasets.data.npolar.no/kartdata/NP_S0_DTM20.zip 
unzip -q *.zip
rm *.zip
find ./ -name *.tif -exec mv {{}} svalbard20.tif \\;

gdalwarp -s_srs EPSG:32633 -t_srs EPSG:3035 -r cubicspline -overwrite -srcnodata 0.0 -dstnodata -9999 svalbard20.tif svalbard20_3035.tif
#gdalwarp -s_srs EPSG:32633 -t_srs EPSG:4326 -r cubicspline -overwrite -srcnodata 0.0 -dstnodata -9999 svalbard20.tif svalbard20_4326.tif

rm -f svalbard20_3035_subset.tif
gdal_translate -projwin {ulx} {uly} {lrx} {lry} svalbard20_3035.tif svalbard20_3035_subset.tif
#gdalwarp -s_srs EPSG:3035 -t_srs EPSG:4326 -r cubicspline -overwrite -srcnodata -3.4028234663852886e+38 -dstnodata -9999 svalbard20_3035_subset.tif svalbard20_4326_subset.tif
#gdal_translate -of "Envi" svalbard20_4326_subset.tif ./LSDTopoTools/Git_projects/LSDTopoTools_ChannelExtraction/driver_functions_ChannelExtraction/svalbard.bil
#gdal_translate -of "AAIGrid" svalbard20_4326_subset.tif ./LSDTopoTools/Git_projects/LSDTopoTools_ChannelExtraction/driver_functions_ChannelExtraction/svalbard.ascii

#Getting rivers
#wget https://nedlasting.geonorge.no/api/download/order/a867cb2b-c1bb-43dc-9939-6f7365754d29/224b3a20-4572-4f79-ad77-440509dabae5-O svalbard.zip
#unzip svalbard.zip #&& mv NP_S100_SOS svalbard

#Getting sosicon
#wget https://github.com/espena/sosicon/blob/master/bin/cmd/linux64/sosicon?raw=true -O sosicon && chmod +x sosicon && mv sosicon ./svalbard/
'''.format(**box)

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
    
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh')
    c.run('rm script.sh')

Creating output file that is 25786P x 36896L.
Processing input file svalbard20.tif.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 25786, 36896
0...10...20...30...40...50...60...70...80...90...100 - done.


## Installing Taudem

In [6]:
svalbardScript = '''#!/bin/bash
mkdir -p taudem
git clone https://github.com/dtarb/TauDEM.git --branch master --single-branch ./taudem
cd taudem/src
mkdir build && cd build
cmake ..
make
sudo make install
cd ~

#Installing lsdtopotools
#git clone https://github.com/LSDtopotools/LSDTopoTools2.git
#cd LSDTopoTools2
#sh lsdtt2_setup.sh

#wget https://raw.githubusercontent.com/LSDtopotools/LSDAutomation/master/LSDTopoToolsSetup.py
#python LSDTopoToolsSetup.py -id 1 -CE True

'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
    
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh',hide='stderr')
    c.run('rm script.sh')

-- The C compiler identification is GNU 6.3.0
-- The CXX compiler identification is GNU 6.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found MPI_C: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so  
-- Found MPI_CXX: /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi_cxx.so;/usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so  
-- Found GDAL: /usr/lib/libgdal.so  
-- Configuring done
-- Generating done
-- Build files have been written to: /home/jose-luis/taudem/src/build
Scanning dependencies of target streamnet
[  1%] Building CXX object CMake

## Pre-processing dem

In [7]:
svalbardScript = '''#!/bin/bash
n=`nproc`
rm fel.tif
rm flow_dir.tif
rm flow_acc.tif
rm streams.tif
rm river*
rm fa.tif

mpiexec -n $n pitremove -z svalbard20_3035_subset.tif -fel fel.tif
mpiexec -n $n d8flowdir -fel fel.tif -p flow_dir.tif
mpiexec -n $n aread8 -p flow_dir.tif -nc -ad8 flow_acc.tif
mpiexec -n $n threshold -ssa flow_acc.tif -thresh 800 -src streams.tif

cp flow_acc.tif fa.tif
gdal_edit.py fa.tif -unsetnodata
gdal_calc.py -A fa.tif --overwrite --outfile=river.tif --calc="A>2500 * logical_not(A==-9999)" --type Byte
gdal_translate -co "NBITS=1" river.tif bit_river.tif

rm fel.tif
rm flow_dir.tif
rm flow_acc.tif
rm streams.tif
rm river.tif
rm fa.tif
rm el.tif

gdal_calc.py -A svalbard20_3035_subset.tif -B bit_river.tif --outfile=el.tif --calc="A-B*10" 
mpiexec -n $n pitremove -z el.tif -fel fel.tif
mpiexec -n $n d8flowdir -fel fel.tif -p flow_dir.tif
mpiexec -n $n aread8 -p flow_dir.tif -nc -ad8 flow_acc.tif

rm bit_river.tif

cp flow_acc.tif fa.tif
gdal_edit.py fa.tif -unsetnodata
gdal_calc.py -A fa.tif --overwrite --outfile=river.tif --calc="A>2500 * logical_not(A==-9999)" --type Byte
gdal_translate -co "NBITS=1" river.tif bit_river.tif

saga_cmd imagery_segmentation "Grid Skeletonization" -INPUT bit_river.tif -METHOD 0 -INIT_METHOD 1 -INIT_THRESHOLD 0 -CONVERGENCE 0 -VECTOR "./river.shp"
'''
#Cloning and installing taudem
with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh', hide='stderr')
    c.run('rm script.sh')

PitRemove version 5.3.8
Input file svalbard20_3035_subset.tif has projected coordinate system.
Processes: 16
Header read time: 0.135610
Data read time: 0.046603
Compute time: 12.114633
Write time: 8.001466
Total time: 20.298312
D8FlowDir version 5.3.8
Input file fel.tif has projected coordinate system.
Processors: 16
Header read time: 0.136489
Data read time: 0.395408
Compute Slope time: 4.714039
Write Slope time: 9.571317
Resolve Flat time: 93.143994
Write Flat time: 3.151290
Total time: 111.112537
AreaD8 version 5.3.8
Input file flow_dir.tif has projected coordinate system.
Number of Processes: 16
Read time: 0.211355
Compute time: 9.848134
Write time: 2.959143
Total time: 13.018632
Threshold version 5.3.8
Input file flow_acc.tif has projected coordinate system.
Compute time: 0.511579
0 .. 10 .. 20 .. 30 .. 40 .. 50 .. 60 .. 70 .. 80 .. 90 .. 100 - Done
Input file size is 12040, 12040
0...10...20...30...40...50...60...70...80...90...100 - done.
0 .. 10 .. 20 .. 30 .. 40 .. 50 .. 60 ..

## Uploading flow_direction and elevation rasters to database


### Setting up procedures on geodatabase

In [8]:
svalbardScript = '''#!/bin/bash
read -r -d '' sql <<-EOM
DROP SCHEMA IF EXISTS procedures CASCADE;
CREATE EXTENSION IF NOT EXISTS plsh;
DROP TYPE IF EXISTS station_info CASCADE;  
CREATE TYPE station_info AS (
station_name varchar(80),
station_id INTEGER,
longitude DOUBLE PRECISION,
latitude DOUBLE PRECISION,
buffer DOUBLE PRECISION,    
epsg INTEGER,
mask TEXT
);
EOM

echo $sql | psql -d geosvalbard
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh')
    c.run('rm script.sh')

DROP SCHEMA


NOTICE:  schema "procedures" does not exist, skipping


CREATE EXTENSION
DROP TYPE


NOTICE:  type "station_info" does not exist, skipping


CREATE TYPE


In [9]:
#The procedures.sql file is downloaded from geonorway: 
#pg_dump --schema=schema_name db_name > backupfile.sql
# and uploaded to this new database
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('/home/jose-luis/Downloads/procedures_oct.sql')
    c.run('psql -d geosvalbard < procedures_oct.sql')
    

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE SCHEMA
ALTER SCHEMA
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION


In [10]:
svalbardScript = '''#!/bin/bash
echo "DROP SCHEMA svalbard CASCADE;" | psql -d geosvalbard
echo "CREATE SCHEMA svalbard;" | psql -d geosvalbard
#Putting rivers in database
shp2pgsql -I -d -s 3035 river.shp svalbard.rivers | psql -q -d geosvalbard
#Putting rasters in database
raster2pgsql -I -M -F -b 1 -r -s 3035 -d -t auto flow_dir.tif  svalbard.flow_dir | psql -q -d geosvalbard
raster2pgsql -I -M -F -b 1 -r -s 3035 -d -t auto svalbard20_3035_subset.tif  svalbard.el | psql -q -d geosvalbard

echo "SELECT procedures.setExtentTable('svalbard','flow_dir');" | psql -d geosvalbard
echo "SELECT procedures.setExtentTable('svalbard','el');" | psql -d geosvalbard

'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh',hide='stderr')
    c.run('rm script.sh')

CREATE SCHEMA
                      addgeometrycolumn                      
-------------------------------------------------------------
 svalbard.rivers.geom SRID:3035 TYPE:MULTILINESTRING DIMS:2 
(1 row)

 setextenttable 
----------------
 
(1 row)

 setextenttable 
----------------
 
(1 row)



### Creating yaml file with stations list

In [34]:
import yaml

coords=[
{'name': 'Adventelva', 'latitude': 78.20388, 'longitude': 15.82857, 'id':1},
{'name': 'Endalselva', 'latitude': 78.200545, 'longitude': 15.816164, 'id':2},
{'name': 'Todalselva', 'latitude': 78.17167, 'longitude': 15.86485, 'id':3},    
{'name': 'Bolterelva', 'latitude': 78.16333, 'longitude': 15.98027, 'id':4},
{'name': 'Foxelva',    'latitude': 78.15919, 'longitude': 16.20452, 'id':5},
{'name': 'AdventelvaSea', 'latitude': 78.2272992, 'longitude': 15.7447349, 'id': 6}, 
{'name': 'DeGeerelva',    'latitude': 78.33822, 'longitude': 16.31238, 'id':7},
{'name': 'Sassenelva',    'latitude': 78.33203, 'longitude': 16.86129, 'id':8},
{'name': 'SassenelvaSea', 'latitude': 78.33648, 'longitude': 16.85598, 'id':9},
{'name': 'Gipsdalselva',  'latitude': 78.44082, 'longitude': 16.57706, 'id':10},
{'name': 'Ebbaelva',      'latitude': 78.70824, 'longitude': 16.599771, 'id':11}
]


#{'name': 'Endalselva', 'latitude': 78.19590, 'longitude': 15.81446, 'id':2},
coords=[{'station': {'station_name': i['name'].strip(),
                        'station_id': i['id'],
                        'longitude': i['longitude'],
                        'latitude': i['latitude'],
                        'epsg': 4326,
                        'buffer': 75000
                       }
          } for i in coords
       ]

with open('adventselva.yaml','w') as f:
    f.write(yaml.dump(coords,allow_unicode=True))

    
def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False 

def infoToString(d):
    l = [ "'"+d[key]+"'" if isinstance(d[key],str) else str(d[key])  for key in ['station_name','station_id','longitude','latitude','buffer','epsg','limits']]
    return 'row(' + ','.join(l) +')::station_info'

def yamlToInfoArray(yamlFile):
    allStations = list()
    stations = yaml.safe_load(open(yamlFile))
    for i in stations:
        data = i['station']
        if is_number(data['buffer']):
            data['limits'] = ''
        else:
            data['buffer'] = -9999
        allStations.append(data)
    return 'ARRAY[' +  ','.join([infoToString(i) for i in allStations]) + ']', ' '.join([str(i['station_id']) for i in allStations]);

array,ids = yamlToInfoArray('adventselva.yaml')
display(array,ids)

"ARRAY[row('Adventelva',1,15.82857,78.20388,75000,4326,'')::station_info,row('Endalselva',2,15.816164,78.200545,75000,4326,'')::station_info,row('Todalselva',3,15.86485,78.17167,75000,4326,'')::station_info,row('Bolterelva',4,15.98027,78.16333,75000,4326,'')::station_info,row('Foxelva',5,16.20452,78.15919,75000,4326,'')::station_info,row('AdventelvaSea',6,15.7447349,78.2272992,75000,4326,'')::station_info,row('DeGeerelva',7,16.31238,78.33822,75000,4326,'')::station_info,row('Sassenelva',8,16.86129,78.33203,75000,4326,'')::station_info,row('SassenelvaSea',9,16.85598,78.33648,75000,4326,'')::station_info,row('Gipsdalselva',10,16.57706,78.44082,75000,4326,'')::station_info,row('Ebbaelva',11,16.599771,78.70824,75000,4326,'')::station_info]"

'1 2 3 4 5 6 7 8 9 10 11'

In [35]:
svalbardScript='''#!/bin/bash
#Initializing schema for basin processing
read -r -d '' SQL <<- EOM
SELECT procedures.initializeStations();
SELECT procedures.addStations({2});
SELECT procedures.initializeResultsSchema('{0}');
SELECT procedures.createDataTable('{0}','dem');
SELECT procedures.createResultsTable('{0}','results');
EOM
echo $SQL | psql -d geosvalbard
# Processing basin for station with station_id id     
for id in {1}
do
  echo $id
  rm -rf Trash${{id}}
  mkdir Trash${{id}}
  cd Trash${{id}}

  #Exporting outlets from database as a shapefile
  pgsql2shp -g outlet -f stations${{id}} geosvalbard "select station_name, station_id, outlet  from {0}.demshp where station_id=${{id}}"
  
  #Computing basing using TauDEM. This produces a boolean tif where true means the cell is upstream of the outlet (i.e. part of the basin)
  n=`nproc`
  mpiexec -n $n gagewatershed -p "PG:dbname=geosvalbard schema={0} table=flow column=rast where='station_id=${{id}}' mode=2" -o stations${{id}}.shp -gw watershed${{id}}.tif

  #Transforming upstream pixels to shapefile
  gdal_polygonize.py -f "ESRI Shapefile" watershed${{id}}.tif basin${{id}}.shp

  #Uploading the basin shapefile to the database
  shp2pgsql -S -d -s 3035 basin${{id}}.shp {0}.dummy | psql -d geosvalbard   
  
  #Placing the basin shapefile in the results table    
  read -r -d '' SQL <<- EOM
      INSERT INTO {0}.resultsShp(station_id,station_name,basin)
      SELECT b.station_id, b.station_name, ST_MakeValid(ST_Multi(ST_Union(a.geom)))
      FROM {0}.stations AS b, {0}.dummy AS a
      WHERE b.station_id=${{id}}
      GROUP BY station_id, station_name;
EOM
  echo $SQL | psql -d geosvalbard

  cd $startDir
  rm -rf Trash${{id}}
done

echo "DROP TABLE IF EXISTS {0}.dummy;" | psql -d geosvalbard

#Getting extent of basin and saving it to a txt file on the server
IFS=$'\\n' read -r -d '' SQL <<- EOM
    (SELECT station_name,Box2D(ST_Transform(St_Buffer(St_Envelope(basin),2000),4326)) FROM {0}.resultsShp)
EOM
echo "\COPY "$SQL" TO '/home/jose-luis/results.txt' DELIMITER ';';" | psql -d geosvalbard
'''.format('test',ids,array)

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh', hide='stderr')
    c.run('rm script.sh')

 initializestations 
--------------------
 
(1 row)

 addstations 
-------------
 
(1 row)

 initializeresultsschema 
-------------------------
 
(1 row)

 createdatatable 
-----------------
 
(1 row)

 createresultstable 
--------------------
 
(1 row)

1
Initializing... 
Done (postgis major version: 2).
Output shape: Point
Dumping: X [1 rows].
Gage Watershed version 5.3.8
Input file PG:dbname=geosvalbard schema=test table=flow column=rast where='station_id=1' mode=2 has projected coordinate system.
Size: 16
Read time: 3.354733
Compute time: 1.657022
Write time: 1.059067
Total time: 6.070822
Creating output basin1.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.
SET
SET
DROP TABLE
BEGIN
CREATE TABLE
ALTER TABLE
               addgeometrycolumn                
------------------------------------------------
 test.dummy.geom SRID:3035 TYPE:POLYGON DIMS:2 
(1 row)

INSERT 0 1
COMMIT
ANALYZE
INSERT 0 1
2
Initializing... 
Done (postgis major versi

### Getting elevation

In [13]:
svalbardScript='''#! /bin/bash
read -r -d '' sql <<-EOM
CREATE OR REPLACE FUNCTION procedures.getElevation(_table text,_station text) RETURNS void AS \$\$ 
DECLARE
  table_name text := 'test.' || _table;
  station  text := _station;
BEGIN 
  EXECUTE 'DROP TABLE IF EXISTS subdivided;';
  EXECUTE FORMAT('CREATE TEMP TABLE subdivided AS 
                  SELECT ST_Subdivide(basin,256) AS geom 
                  FROM test.resultsshp AS a 
                  WHERE a.station_name=%s;', \'\'\'\' || station || \'\'\'\' );
  EXECUTE 'DROP INDEX IF EXISTS subdivided_idx';
  EXECUTE 'CREATE INDEX subdivided_idx ON subdivided USING GIST(geom)';
  
  EXECUTE 'DROP TABLE IF EXISTS elevData';
  EXECUTE FORMAT('CREATE TEMP TABLE elevData AS 
                  SELECT ST_DumpAsPolygons(rast) AS geo 
                  FROM test.resultsshp AS a, svalbard.el as b 
                  WHERE ST_Intersects(b.extent,a.basin) 
                  AND a.station_name=%s',\'\'\'\' || station || \'\'\'\');
  EXECUTE 'DROP INDEX IF EXISTS elevData_idx;';
  EXECUTE 'CREATE INDEX elevData_idx ON elevData USING GIST(((elevData.geo).geom));';
  
  EXECUTE FORMAT('DROP TABLE IF EXISTS %s;',table_name);
  EXECUTE FORMAT('CREATE TABLE %s(id SERIAL PRIMARY KEY, elev DOUBLE PRECISION, geom GEOMETRY(POLYGON,3035));', table_name);
  EXECUTE FORMAT('INSERT INTO %s(elev,geom) 
                  SELECT DISTINCT (a.geo).val, (a.geo).geom 
                  FROM elevData AS a, subdivided AS b 
                  WHERE ST_Intersects(b.geom,(a.geo).geom);', table_name);
  
  EXECUTE 'DROP TABLE IF EXISTS toRemove;';
  EXECUTE 'CREATE TEMP TABLE toRemove(id INTEGER, geom GEOMETRY(POLYGON,3035));';
  EXECUTE FORMAT('WITH myLine AS (SELECT ST_Boundary(basin) AS geom FROM test.resultsshp AS a WHERE a.station_name=%s) 
                  INSERT INTO toRemove(id,geom)
                  SELECT a.id,a.geom 
                  FROM %s AS a, myLine AS b
                  WHERE ST_Intersects(a.geom,b.geom);', \'\'\'\' || station || \'\'\'\', table_name);
  
  EXECUTE 'DROP TABLE IF EXISTS intersections;';
  EXECUTE 'CREATE TEMP TABLE intersections AS
           SELECT a.id, SUM(ST_Area(ST_Intersection(a.geom,b.geom))) AS suma
           FROM subdivided AS b, toRemove AS a
           WHERE ST_Intersects(a.geom,b.geom)
           GROUP by a.id;';
  
  EXECUTE FORMAT('DELETE FROM %s AS A USING intersections AS b WHERE a.id=b.id AND b.suma < 0.1;', table_name);
  
  RETURN;
END;
\$\$ LANGUAGE PLPGSQL;
EOM

echo $sql | psql -d geosvalbard
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh')
    c.run('rm script.sh')

CREATE FUNCTION


In [14]:
import pandas as pd
import numpy as np


stations = yaml.safe_load(open('adventselva.yaml'))
def cdf(x, plot=True, *args, **kwargs):
    x, y = (sorted(x) - min(x)) / (max(x) - min(x)), np.arange(len(x)) / len(x)
    #plt.plot(y, x, *args, **kwargs) if plot else (x, y)
    #plt.xlim([1.01,-0.01])
    return x,y

def getElevation(table,station):
    svalbardScript='''#! /bin/bash
echo "SELECT procedures.getElevation(\'{0}\',\'{1}\');" | psql -d geosvalbard
echo "\COPY (SELECT elev from test.{0}) TO \'/home/jose-luis/dummy.txt\' DELIMITER \',\';" | psql -d geosvalbard
'''.format(table,station)
   
    with open('script.sh','w') as bla:
        bla.write(svalbardScript)
    with Connection(host=basin.properties['ip'],
                   user=basin.properties['username'],
                   connect_kwargs={"key_filename": basin.properties['keyFile']},
                   ) as c:
        c.put('script.sh')
        c.run('chmod +x script.sh')
        c.run('./script.sh',hide='stderr')
        #c.run('rm script.sh')
        c.get('dummy.txt')
    elevation = pd.read_csv('dummy.txt'.format(station), header=None, names=['Elevation',])
    return elevation

cnt = False
ecdf = dict()
for i in stations:
    station = i['station']['station_name']
    elevation = getElevation(station + 'Elev',station)   
    x,y=cdf(elevation.Elevation.values)
    ecdf[station]= {'x': x, 'y': y}
    elevation.rename(columns={'Elevation':station},inplace=True)
    if not cnt:
        info = elevation.describe()
        cnt = True
    else:
        dummy = elevation.describe()
        info = pd.concat([info, dummy], axis=1, join='inner')

        
display(info)
info.to_csv('elevationInfo.txt')

NOTICE:  table "subdivided" does not exist, skipping
NOTICE:  index "subdivided_idx" does not exist, skipping
NOTICE:  table "elevdata" does not exist, skipping
NOTICE:  index "elevdata_idx" does not exist, skipping
NOTICE:  table "adventelvaelev" does not exist, skipping
NOTICE:  table "toremove" does not exist, skipping
NOTICE:  table "intersections" does not exist, skipping
NOTICE:  table "subdivided" does not exist, skipping
NOTICE:  index "subdivided_idx" does not exist, skipping
NOTICE:  table "elevdata" does not exist, skipping
NOTICE:  index "elevdata_idx" does not exist, skipping
NOTICE:  table "endalselvaelev" does not exist, skipping
NOTICE:  table "toremove" does not exist, skipping
NOTICE:  table "intersections" does not exist, skipping
NOTICE:  table "subdivided" does not exist, skipping
NOTICE:  index "subdivided_idx" does not exist, skipping
NOTICE:  table "elevdata" does not exist, skipping
NOTICE:  index "elevdata_idx" does not exist, skipping
NOTICE:  table "todalsel

,Adventelva,Endalselva,Todalselva,Bolterelva,Foxelva,AdventelvaSea,DeGeerelva,Sassenelva,SassenelvaSea,Gipsdalselva,Ebbaelva
count,1.079158e+06,77331.000000,90592.000000,92943.000000,64714.000000,1.280957e+06,204311.000000,22857.000000,1.825019e+06,578839.000000,137854.000000
mean,4.393804e+02,452.164753,484.637470,508.335654,546.297248,4.335712e+02,412.128910,76.982761,3.430105e+02,386.728876,504.494462
std,2.500534e+02,239.056164,204.604542,205.193617,206.496521,2.532794e+02,217.572433,97.399792,2.226645e+02,262.005384,258.190201
min,3.224963e+00,3.983034,49.152260,56.953365,98.371750,7.024645e-01,9.694341,0.516186,5.108159e-01,0.902945,0.439645
25%,2.226992e+02,270.084702,351.762215,363.802490,393.475090,2.156571e+02,234.671982,3.109151,1.333124e+02,131.576912,325.537239
50%,4.483418e+02,478.571259,500.938812,527.569885,562.466034,4.457000e+02,404.384613,24.780209,3.652377e+02,396.914703,556.465576
75%,6.376409e+02,610.771790,603.768387,669.581848,717.590591,6.318264e+02,570.898193,157.143204,5.276397e+02,593.961700,692.780731
max,1.143694e+03,1020.984863,1021.318420,966.332947,1012.748657,1.143694e+03,991.560669,477.400085,9.335325e+02,1117.072998,1119.646973


In [15]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for key, values in ecdf.items(): 
    plt.plot(values['y'],values['x'],label=key)
    
plt.xlim([1.01,-0.01])
plt.legend(loc='upper right')

plt.savefig('hypsogram.png')


In [36]:
a = query('''SELECT json_build_object('type', 'FeatureCollection',
                                      'features', json_agg(json_build_object(
                                                                'type',       'Feature',
                                                                'label',      station_name,
                                                                'geometry',   ST_AsGeoJSON(ST_ForceRHR(St_Transform(basin,4326)))::json,
                                                                'properties', jsonb_set(row_to_json(resultsShp)::jsonb,'{basin}','0',false)
                                                                )
                                                            )
                                     )
             FROM test.resultsShp;''')

fig = gmaps.figure(map_type="TERRAIN")
fig.add_layer(gmaps.geojson_layer(a[0][0]))

b = query('''SELECT a.station_name, st_x(st_transform(a.outlet,4326)),
    st_y(st_transform(a.outlet,4326)), st_area(b.basin)/1e6
    FROM test.demShp AS a
    INNER JOIN test.resultsShp AS b 
    ON a.station_id = b.station_id''')

outlets = [{"name": i[0], "area": i[3]} for i in b]
locations = [(float(i[2]),float(i[1])) for i in b]
info_box_template = """
<dl>
<font color="black">
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</font>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))

## Getting shapefile

In [17]:
svalbardScript='''#! /bin/bash
pgsql2shp -f "./svalbard_basins.shp" -h localhost geosvalbard "SELECT *, st_area(basin) as  basinArea FROM test.resultsshp;"
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh')
    c.run('tar -cf  svalbardShp.tar svalbard_basins.*')
    c.get('svalbardShp.tar')
    c.run('rm svalbard_basins*')
    c.run('rm script.sh')

Initializing... 
Done (postgis major version: 2).
Output shape: Polygon
Dumping: X [11 rows].


## Intersecting glaciers

In [18]:
with Connection('localhost') as c:
    c.local('cp /home/jose-luis/Envs/niva_prognos/PROGNOS/notebooks/createGeodatabase/Shapefiles/NP_S100_SHP/S100_Isbreer_f.* .')
    c.local('tar -cf glaciers.tar S100_Isbreer_f.*')
    c.local('rm -r ./S100_Isbreer_f.*')


svalbardScript='''#! /bin/bash
shp2pgsql -I -s 32633:3035 -S -W "LATIN1"  -d S100_Isbreer_f.shp test.glaciers | psql -q -d geosvalbard
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.put('glaciers.tar')
    c.run('tar -xf glaciers.tar')
    c.run('rm glaciers.tar')
    c.run('./script.sh')
    #c.run('tar -cf  svalbardShp.tar S100_Isbreer_f.*')
    c.run('rm S100_Isbreer_f.*')
    c.run('rm script.sh')

Shapefile type: PolygonZ
Postgis type: POLYGON[4]
ERROR:  column not found in geometry_columns table
CONTEXT:  PL/pgSQL function dropgeometrycolumn(character varying,character varying,character varying,character varying) line 34 at RAISE
SQL statement "SELECT DropGeometryColumn('',$1,$2,$3)"
PL/pgSQL function dropgeometrycolumn(character varying,character varying,character varying) line 5 at SQL statement
NOTICE:  table "glaciers" does not exist, skipping


                 addgeometrycolumn                 
---------------------------------------------------
 test.glaciers.geom SRID:3035 TYPE:POLYGON DIMS:4 
(1 row)



In [19]:

query("CREATE INDEX basins_idx ON test.resultsshp USING GIST(basin);",fetch=False)


In [20]:
display(query('''CREATE TEMP TABLE dummy AS SELECT a.gid,b.station_name,st_intersection(a.geom,b.basin) as glacierCover
FROM test.glaciers AS a, test.resultsshp AS b WHERE ST_Intersects(a.geom,b.basin);
CREATE TEMP TABLE dummy_accum AS SELECT c.station_name,sum(st_area(c.glacierCover))/1e6 as glacierArea FROM dummy AS c
GROUP BY station_name;
SELECT a.station_name, a.glacierArea, st_area(b.basin)/1e6, a.glacierArea/(st_area(b.basin)/1e6) * 100 FROM dummy_accum AS a
INNER JOIN test.resultsshp as b
ON a.station_name = b.station_name;
'''))


[('Ebbaelva', 28.1945882277367, 54.8167380951996, 51.4342684505808),
 ('Endalselva', 4.24358132687086, 30.7312759240085, 13.8086727585417),
 ('AdventelvaSea', 95.9645529638512, 510.488696498593, 18.7985656924562),
 ('Foxelva', 8.68756054858029, 25.7182064887003, 33.7798071276755),
 ('SassenelvaSea', 195.324008319055, 725.508685806743, 26.9223528456949),
 ('Todalselva', 3.73765231976206, 36.004217538349, 10.3811513631174),
 ('Gipsdalselva', 40.6649097931255, 230.132708857379, 17.6701999446445),
 ('DeGeerelva', 8.38424196845165, 81.1945589832831, 10.3261130714163),
 ('Bolterelva', 7.87480665634056, 37.0119176087329, 21.2764081547683),
 ('Adventelva', 87.1195915960227, 428.961151451043, 20.3094362511206)]

## Getting geology

In [31]:
with Connection('localhost') as c:
    c.local('cp /home/jose-luis/Envs/niva_prognos/PROGNOS/notebooks/createGeodatabase/Shapefiles/Geology_data/Svalbard_Geology_G250/G250_Shape_files/g250_Geology.* .')
    c.local('tar -cf geology.tar g250_Geology.*')
    c.local('rm -r ./g250_Geology.*')


svalbardScript='''#! /bin/bash
shp2pgsql -I -s 32633:3035 -W "LATIN1"  -d g250_Geology.shp test.geology | psql -q -d geosvalbard
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.put('geology.tar')
    c.run('tar -xf geology.tar')
    c.run('rm geology.tar')
    c.run('./script.sh')
    c.run('rm g250_Geology.*')
    c.run('rm script.sh')
    
#query("CREATE INDEX geology_idx ON test.geology USING GIST(geom);",fetch=False)


Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]
ERROR:  column not found in geometry_columns table
CONTEXT:  PL/pgSQL function dropgeometrycolumn(character varying,character varying,character varying,character varying) line 34 at RAISE
SQL statement "SELECT DropGeometryColumn('',$1,$2,$3)"
PL/pgSQL function dropgeometrycolumn(character varying,character varying,character varying) line 5 at SQL statement
NOTICE:  table "geology" does not exist, skipping


                   addgeometrycolumn                   
-------------------------------------------------------
 test.geology.geom SRID:3035 TYPE:MULTIPOLYGON DIMS:2 
(1 row)



In [4]:
import pandas as pd
a = query('''CREATE TEMP TABLE dummy AS SELECT a.main_litho,b.station_name,st_intersection(a.geom,b.basin) as geologyCover
FROM test.geology AS a, test.resultsshp AS b WHERE ST_Intersects(a.geom,b.basin);
CREATE TEMP TABLE dummy_accum AS SELECT c.station_name,c.main_litho,sum(st_area(c.geologyCover))/1e6 as geologyArea FROM dummy AS c
GROUP BY station_name, main_litho
ORDER BY station_name,geologyArea;
SELECT a.station_name, a.main_litho, a.geologyArea, a.geologyArea/(st_area(b.basin)/1e6) * 100 FROM dummy_accum AS a
INNER JOIN test.resultsshp as b
ON a.station_name = b.station_name;
''')

name = [i[0] for i in a]
main_litho = [i[1] for i in a]
area = [i[2] for i in a]
percentage = [i[3] for i in a]

data = [name,main_litho,area,percentage]

data =  pd.DataFrame({'station_name' : name,'main_litho' :  main_litho, 'area(km2)' : area, 'percentage' : percentage})
display(data)
data.to_csv('geology.csv')


,station_name,main_litho,area(km2),percentage
0,Adventelva,"sandstone, siltstone, mudstone",3.482634,0.811876
1,Adventelva,marine deposit,3.667394,0.854948
2,Adventelva,"sandstone, siltstone, shale",4.791403,1.116978
3,Adventelva,"sandstone, shale, mudstone, conglomerate",7.173016,1.672183
4,Adventelva,"shale, mudstone, siltstone",9.213427,2.147847
5,Adventelva,"shale, sandstone",13.164291,3.068877
6,Adventelva,moraine,16.269962,3.792875
7,Adventelva,"sandstone, shale, coal",16.687749,3.890270
8,Adventelva,"sandstone, conglomerate",23.633154,5.509393
9,Adventelva,sandstone,31.031797,7.234174


## Computing slope

In [21]:
svalbardScript='''#! /bin/bash
rm -rf slope.tif
gdaldem slope svalbard20_3035_subset.tif slope.tif -zero_for_flat
raster2pgsql -I -M -F -b 1 -r -s 3035 -d -t auto slope.tif  svalbard.slope | psql -q -d geosvalbard
'''

with open('script.sh','w') as bla:
    bla.write(svalbardScript)
with Connection(host=basin.properties['ip'],
                       user=basin.properties['username'],
                       connect_kwargs={"key_filename": basin.properties['keyFile']},
                       ) as c:
    c.put('script.sh')
    c.run('chmod +x script.sh')
    c.run('./script.sh', hide='stderr')
    c.run('rm script.sh')

0...10...20...30...40...50...60...70...80...90...100 - done.


In [13]:
query("SELECT procedures.setExtentTable('svalbard','slope');",fetch=False);  #todo: looks like this only works as user postgres
display(query('''select * from svalbard.slope where false;'''))

[]

In [32]:
stations = [i[0] for i in query("select station_name from test.resultsshp;")]

queryStr = '''CREATE TEMP TABLE subdivided AS 
SELECT ST_Subdivide(basin,256) AS geom 
FROM test.resultsshp AS a 
WHERE a.station_name='{0}';
DROP INDEX IF EXISTS subdivided_idx;
CREATE INDEX subdivided_idx ON subdivided USING GIST(geom);
DROP TABLE IF EXISTS slopeData;
CREATE TEMP TABLE slopeData AS 
SELECT ST_DumpAsPolygons(rast) AS geo 
FROM test.resultsshp AS a, svalbard.slope as b 
WHERE ST_Intersects(b.extent,a.basin) 
AND a.station_name='{0}';
DROP INDEX IF EXISTS slopeData_idx;
CREATE INDEX slopeData_idx ON slopeData USING GIST(((slopeData.geo).geom));
CREATE TEMP TABLE {0}(id SERIAL PRIMARY KEY, slope DOUBLE PRECISION, geom GEOMETRY(POLYGON,3035));
INSERT INTO {0}(slope,geom)
SELECT DISTINCT (a.geo).val, (a.geo).geom 
FROM slopeData AS a, subdivided AS b 
WHERE ST_Intersects(b.geom,(a.geo).geom);
CREATE TEMP TABLE toRemove(id INTEGER, geom GEOMETRY(POLYGON,3035));
WITH myLine AS (SELECT ST_Boundary(basin) AS geom FROM test.resultsshp AS a WHERE a.station_name='{0}') 
                  INSERT INTO toRemove(id,geom)
                  SELECT a.id,a.geom 
                  FROM {0} AS a, myLine AS b
                  WHERE ST_Intersects(a.geom,b.geom);
CREATE TEMP TABLE intersections AS
SELECT a.id, SUM(ST_Area(ST_Intersection(a.geom,b.geom))) AS suma
                 FROM subdivided AS b, toRemove AS a
                 WHERE ST_Intersects(a.geom,b.geom)
                 GROUP by a.id;
DELETE FROM {0} AS a USING intersections AS b WHERE a.id=b.id AND b.suma < 0.1;
SELECT slope,st_area(geom) FROM {0};
'''
cnt = 0
for i in stations:
    display(i)
    data = query(queryStr.format(i))
    dummy = pd.DataFrame({i : [j[0] for j in data]}).describe()
    if cnt == 0:
        result = dummy;
    else :
        result = pd.concat([result, dummy], axis=1, join='inner')
    cnt=cnt+1

display(result)

'Adventelva'

'Endalselva'

'Todalselva'

'Bolterelva'

'Foxelva'

...

'Sassenelva'

'SassenelvaSea'

'Gipsdalselva'

'Ebbaelva'

,Adventelva,Endalselva,Todalselva,Bolterelva,Foxelva,AdventelvaSea,DeGeerelva,Sassenelva,SassenelvaSea,Gipsdalselva,Ebbaelva
count,1.079368e+06,77338.000000,90605.000000,93051.000000,64722.000000,1.281852e+06,204333.000000,22857.000000,1.825690e+06,579092.000000,137946.000000
mean,1.539780e+01,16.522654,18.511765,17.524826,18.785330,1.575044e+01,15.632703,6.278237,1.161848e+01,19.544401,14.075467
std,1.085463e+01,11.657729,10.883319,11.529108,11.080920,1.116323e+01,9.720529,8.562740,1.056357e+01,13.483522,11.700644
min,0.000000e+00,0.006877,0.048183,0.000000,0.099983,0.000000e+00,0.008896,0.001473,0.000000e+00,0.000000,0.000000
25%,6.237889e+00,6.120664,8.784490,7.258502,9.366005,6.221559e+00,7.710014,0.283197,3.306561e+00,6.287843,4.932989
50%,1.310797e+01,14.220090,18.177288,15.489082,17.268409,1.349174e+01,14.327744,3.976148,7.900057e+00,19.480540,8.863428
75%,2.372712e+01,26.228646,27.238571,27.542756,27.959727,2.468103e+01,22.497789,8.038993,1.796215e+01,31.954687,22.870623
max,5.511977e+01,55.415203,52.058754,52.482899,54.593193,5.720966e+01,46.077187,45.662560,6.376051e+01,64.810524,70.977844


In [33]:
result.to_csv('slope_in_degrees.csv')